# Spam Classification Using SVM,Logistic regression, NN and Decision tree

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/spam.csv


# Data Reading

In [2]:
data_spam=pd.read_csv('/kaggle/input/spam.csv',encoding='latin-1')
data_spam=data_spam.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

# Data text Preprocess function

In [3]:
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

def pre_process(text):    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

# Importing models

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn import tree

# Initiating models

In [5]:
lr=LogisticRegression()
svm=SVC(kernel='rbf')
nn=NearestNeighbors(n_neighbors=2, algorithm='ball_tree')
dt=tree.DecisionTreeClassifier()

# Pre processing Data and splitting

In [6]:
X=data_spam['v2']
y=data_spam['v1'].map({'ham':0,'spam':1})
X = X.apply(pre_process)
vectorizer = TfidfVectorizer("english")
X = vectorizer.fit_transform(X)
X_train,X_test,Y_train,Y_test=train_test_split(X, y, test_size = 0.2, random_state = 0)

# Training Data on all 4 models

In [7]:
lr.fit(X_train,Y_train)
svm.fit(X_train,Y_train)
nn.fit(X_train,Y_train)
dt.fit(X_train,Y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/neighbors/base.py:216: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

# Prediction step

In [8]:
lr_predicted=lr.predict(X_test)
svm_predicted=svm.predict(X_test)
nn_predicted=nn.kneighbors(X_test)
dt_predicted=dt.predict(X_test)

# Classification error metric

In [9]:
lr_cls_report=classification_report(Y_test,lr_predicted)
svm_cls_report=classification_report(Y_test,svm_predicted)
# nb_cls_report=classification_report(Y_test,nb_predicted)
dt_cls_report=classification_report(Y_test,dt_predicted)
print("Logistic regression : "+lr_cls_report)
print("SVM : "+svm_cls_report)
# print("Naive Bayes : "+nb_cls_report)
print("Decision tree : "+dt_cls_report)

Logistic regression :               precision    recall  f1-score   support

           0       0.94      1.00      0.97       949
           1       0.99      0.61      0.76       166

    accuracy                           0.94      1115
   macro avg       0.96      0.81      0.86      1115
weighted avg       0.94      0.94      0.94      1115

SVM :               precision    recall  f1-score   support

           0       0.85      1.00      0.92       949
           1       0.00      0.00      0.00       166

    accuracy                           0.85      1115
   macro avg       0.43      0.50      0.46      1115
weighted avg       0.72      0.85      0.78      1115

Decision tree :               precision    recall  f1-score   support

           0       0.97      0.99      0.98       949
           1       0.91      0.83      0.86       166

    accuracy                           0.96      1115
   macro avg       0.94      0.91      0.92      1115
weighted avg       0.96      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## The LR model has performed the best among the four algos. Ironically Naive bayes is supposed to perofrm the best as per literature. Initially I have run this on AWS sagemaker and found that NB algo requires more memory power. So, considering the computational, accuracy and memory requirements, LR performs the best.